In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
# from typing import Any
# np.set_printoptions(precision=5)
from utils import load_gpt2_params_from_tf_ckpt
from utils import load_weights_into_gpt
# from utils import print_layer_structure
from utils import tldr

from utils import generate_text_simple
from utils import text_to_token_ids
from utils import token_ids_to_text

In [2]:
from gpt2t import GPT2t

In [2]:
# class SelfAttention(tf.keras.layers.Layer):
#     def __init__(self, d_in, d_out, context_length, dropout, num_heads, b, use_outproj=False): # , use_outproj=False):
#         super().__init__(name="self")
#         self.debug = False
#         assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
#         qkv_bias=True # ?
#         self.d_out = d_out
#         self.num_heads = num_heads
#         self.head_dim = d_out // num_heads
#         self.query_layer = tf.keras.layers.Dense(units=d_out, activation=None, name=f"query-{b}", use_bias=qkv_bias)
#         self.query_layer.build((None, d_in))
#         self.key_layer = tf.keras.layers.Dense(units=d_out, activation=None, name=f"key-{b}", use_bias=qkv_bias)
#         self.key_layer.build((None, d_in))
#         self.value_layer = tf.keras.layers.Dense(units=d_out, activation=None, name=f"value-{b}", use_bias=qkv_bias)
#         self.value_layer.build((None, d_in))
#         self.use_outproj = use_outproj

#         self.out_proj = tf.keras.layers.Dense(units=d_out, activation=None, name=f"proj-{b}", use_bias=True)
#         self.out_proj.build((None, d_out))

#         mask = tf.ones((context_length, context_length), dtype=tf.bool) # square matrix of True
#         causal_mask = tf.linalg.band_part(mask, num_lower=-1, num_upper=0) # upper right becomes False
#         additive_mask = 1.0 - tf.cast(causal_mask, dtype=tf.float32) # upper right becomes 1.0
#         self.additive_mask_applied = additive_mask * -1e9   # upper right is large negative value
        
#     def __call__(self, inputs):
#         if self.debug:
#             print(f".... input to    SelfAttention: {tldr(inputs)}")
#         batch_size, num_tokens, d_in = inputs.shape

#         keys = self.key_layer(inputs)      
#         queries = self.query_layer(inputs)
#         values = self.value_layer(inputs)
        
#         keys = tf.reshape(keys, [batch_size, num_tokens, self.num_heads, self.head_dim ]) # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim): 2, 6, 2 -> 2, 6, 2, 1        
#         queries = tf.reshape(queries, [batch_size, num_tokens, self.num_heads, self.head_dim ]) # 2, 6, 2, 1
#         values = tf.reshape(values, [batch_size, num_tokens, self.num_heads, self.head_dim ]) # 2, 6, 2, 1

#         keys = tf.transpose(keys, perm=[0, 2, 1, 3])      # [2,6,2,1] ->  [2, 2, 6, 1]        
#         values = tf.transpose(values, perm=[0, 2, 1, 3])      # [2,6,2,1] ->  [2, 2, 6, 1]
#         queries = tf.transpose(queries, perm=[0, 2, 1, 3])      # [2,6,2,1] ->  [2, 2, 6, 1]
        
#         attn_scores = tf.matmul(queries, tf.transpose(keys, perm=[0, 1, 3, 2]))
#         trimmed_additive_mask_applied = self.additive_mask_applied[:num_tokens, :num_tokens]
#         attn_scores = attn_scores + trimmed_additive_mask_applied
#         attn_weights = tf.nn.softmax(attn_scores / keys.shape[-1]**0.5, axis=-1)

#         context_vec = tf.matmul(attn_weights, values)        
#         context_vec = tf.transpose(context_vec, perm=[0, 2, 1, 3])
#         context_vec = tf.reshape(context_vec, [batch_size, num_tokens, self.d_out]) # (b, num_tokens, self.d_out)

#         if self.use_outproj: # llmfs uses this!
#             context_vec = self.out_proj(context_vec)
#         if self.debug:
#             print(f".... output from SelfAttention: {tldr(context_vec)}")
#         return context_vec

In [3]:
# class AttentionLayer(tf.keras.layers.Layer):
#     def __init__(self, d_in, d_out, context_length, dropout, num_heads, b):
#         super().__init__(name="attention")
#         self.layer_norm = tf.keras.layers.LayerNormalization(name="layer_norm") 
#         self.self_attention = SelfAttention(d_in, d_out, context_length, dropout, num_heads, b)
#         self.projection = tf.keras.layers.Dense(units=d_out, activation=None, name=f"projection")
#         self.debug = False
    
#     def __call__(self, inputs):
#         if self.debug:
#             print(f".... input to    layer_norm: {tldr(inputs)}")
#         x = self.layer_norm(inputs)
#         if self.debug:
#             print(f".... output from layer_norm: {tldr(x)}")
#         x = self.self_attention(x)
#         if self.debug:
#             print(f".... input to    projection: {tldr(x)}")
#         x = self.projection(x)
#         if self.debug:
#             print(f".... output from projection: {tldr(x)}")
#         return x

In [4]:
# class MultiLayerPerceptron(tf.keras.layers.Layer):
#     def __init__(self, d_out, b):
#         super().__init__(name="mlp")
#         self.layer_norm = tf.keras.layers.LayerNormalization(name=f"layer_norm")
#         self.perceptron = tf.keras.layers.Dense(units=d_out * 4, activation=tf.keras.activations.gelu, name=f"perceptron")
#         self.projection = tf.keras.layers.Dense(units=d_out, name=f"projection")
#         self.debug = False
#     def __call__(self, inputs):
#         if self.debug:
#             print(f".... input to    mlp: {tldr(inputs)}")
#         x = self.layer_norm(inputs)
#         x = self.perceptron(x)
#         x = self.projection(x)
#         if self.debug:
#             print(f".... output from mlp: {tldr(x)}")
#         return x

In [5]:
# class Block(tf.keras.layers.Layer):
#     def __init__(self, d_in, d_out, context_length, dropout, num_heads, b, qkv_bias=False):
#         super().__init__(name=f'block-{b}')
#         self.b = b       

#         self.attention = AttentionLayer(d_in, d_out, context_length, dropout, num_heads, b)
#         self.mlp = MultiLayerPerceptron(d_out, b)
#         self.debug = False
#     def __call__(self, inputs):
#         if self.debug:
#             print()
#             print(f".. input to    block {self.b}: {tldr(inputs)}")
#         x = inputs
#         a = self.attention(x)
#         x = x + a
#         m = self.mlp(x)
#         x = x + m
#         if self.debug:
#             print(f".. output from block {self.b}: {tldr(x)}")
#         return x

In [6]:
# class Transformer(tf.keras.layers.Layer):
#     def __init__(self, blocks_num, d_in, d_out, context_length, dropout, num_heads):
#         super().__init__(name="transformer")
#         self.blocks_num = blocks_num
#         self.blocks = []
#         for b in range(blocks_num):
#             block = Block(d_in, d_out, context_length, dropout, num_heads, b)
#             self.blocks.append(block)
#         self.layer_norm = tf.keras.layers.LayerNormalization(name=f"layer_norm")
             
#     def __call__(self, inputs):
#         x = inputs
#         for b in range(self.blocks_num):
#             x = self.blocks[b](x)

#         x = self.layer_norm(x)
#         return x

In [7]:
# class Embedding(tf.keras.layers.Layer):
#     def __init__(self, embedding_size, vocab_size, max_position_length, dtype=tf.float32):
#         super().__init__(name="embedding", dtype=dtype)
#         self.embedding_size = embedding_size
#         self.vocab_size = vocab_size
#         self.max_position_length = max_position_length
#         self.word_embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, name="word_embedding")
#         self.word_embedding.build((None, self.embedding_size))
#         self.position_embedding = tf.keras.layers.Embedding(input_dim=self.max_position_length, output_dim=self.embedding_size, name="position_embedding")
#         self.position_embedding.build((None, self.embedding_size))
#         self.debug = False
#     def __call__(self, inputs):
#         we = self.word_embedding(inputs)
#         pe = self.position_embedding(tf.range(1024))
#         pe_corrected = pe[:we.shape[1], :]
#         x = we + pe_corrected
#         if self.debug:
#             print(f".. Embedding output: {tldr(x)}")
#         return x

In [8]:
# class GPT2t(tf.keras.Model):

#     def __init__(self, config, name=None, trainable=True, dtype=None):
#         super().__init__(name=name)
#         self.trainable = trainable
#         self.embedding_size=config['n_embd']
#         self.vocab_size=config['n_vocab']
#         self.max_position_length=config['n_ctx']
#         self.blocks_num = config["n_layer"]
#         d_in=config['n_embd']
#         d_out=config['n_embd']
#         context_length = config['n_ctx']
#         num_heads = config['n_head']
#         self.embedding = Embedding(embedding_size=self.embedding_size, vocab_size=self.vocab_size, max_position_length=self.max_position_length)
#         self.transformer = Transformer(self.blocks_num, d_in, d_out, context_length, dropout=None, num_heads=num_heads)
#         self.debug = False
        
#     def __call__(self, inputs):
#         x = self.embedding(inputs)
#         x = self.transformer(x)
#         if self.debug:
#             print(f".. input to final matmul: {tldr(x)}")
#         logits = tf.matmul(x, self.embedding.word_embedding.get_weights()[0], transpose_b=True)
#         if self.debug:
#             print(f".. final logits: {tldr(logits)}")
#         return logits

In [3]:
config124M = {'n_embd': 768,  'n_vocab': 50257, 'n_ctx': 1024, 'n_layer': 12, 'n_head': 12}
# config355M = {'n_embd': 1024, 'n_vocab': 50257, 'n_ctx': 1024, 'n_layer': 24, 'n_head': 16}
gpt2t=GPT2t(config124M)

In [4]:
# model_dir="ch05/01_main-chapter-code/gpt2/124M"
model_dir="gpt2/124M"
tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, config124M)

In [10]:
params.keys()

dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])

In [68]:
params['g'].shape[0]

1024

In [69]:
params['blocks'][0]['ln_1']['b'].shape

(1024,)

In [70]:
params["blocks"][0]["mlp"]["c_proj"]["w"].shape[0]

4096

### Now make a function to load the weights

In [12]:
# def load_weights_into_gpt(gpt, params):
#     n_embd = params['g'].shape[0]
#     _ = gpt.embedding(tf.constant([[1]]))
#     assert gpt.embedding.word_embedding.built
#     assert gpt.embedding.position_embedding.built
#     gpt.embedding.word_embedding.set_weights([np.array(params['wte'])])
#     gpt.embedding.position_embedding.set_weights([np.array(params['wpe'])])
#     for b in range(len(params["blocks"])):
#         gpt.transformer.blocks[b].attention.layer_norm.beta_initializer = tf.keras.initializers.Constant(params["blocks"][b]["ln_1"]["b"]) 
#         gpt.transformer.blocks[b].attention.layer_norm.gamma_initializer = tf.keras.initializers.Constant(params["blocks"][b]["ln_1"]["g"]) 
#         gpt.transformer.blocks[b].attention.layer_norm.build((None, None, n_embd))
#         assert gpt.transformer.blocks[b].attention.layer_norm.built

#         q_w, k_w, v_w = np.split((params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
#         q_b, k_b, v_b = np.split((params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        
#         assert gpt.transformer.blocks[b].attention.self_attention.query_layer.built
#         gpt.transformer.blocks[b].attention.self_attention.query_layer.set_weights([q_w, q_b])
#         assert gpt.transformer.blocks[b].attention.self_attention.key_layer.built
#         gpt.transformer.blocks[b].attention.self_attention.key_layer.set_weights([k_w, k_b])
#         assert gpt.transformer.blocks[b].attention.self_attention.value_layer.built
#         gpt.transformer.blocks[b].attention.self_attention.value_layer.set_weights([v_w, v_b])
    
#         # AttentionLayer projection
#         gpt.transformer.blocks[b].attention.projection.build((None, n_embd))
#         assert gpt.transformer.blocks[b].attention.projection.built
#         gpt.transformer.blocks[b].attention.projection.set_weights([params["blocks"][b]["attn"]["c_proj"]["w"], params["blocks"][b]["attn"]["c_proj"]["b"]])      
    
#         # MultiLayerPerceptron layer_norm
#         gpt.transformer.blocks[b].mlp.layer_norm.beta_initializer = tf.keras.initializers.Constant(params["blocks"][b]["ln_2"]["b"]) 
#         gpt.transformer.blocks[b].mlp.layer_norm.gamma_initializer = tf.keras.initializers.Constant(params["blocks"][b]["ln_2"]["g"])
#         gpt.transformer.blocks[b].mlp.layer_norm.build((None, None, n_embd))
#         assert gpt.transformer.blocks[b].mlp.layer_norm.built
    
#         # MultiLayerPerceptron perceptron
#         gpt.transformer.blocks[b].mlp.perceptron.build((None, n_embd))
#         assert gpt.transformer.blocks[b].mlp.perceptron.built
#         gpt.transformer.blocks[b].mlp.perceptron.set_weights([params["blocks"][b]["mlp"]["c_fc"]["w"], params["blocks"][b]["mlp"]["c_fc"]["b"]])
#         # MultiLayerPerceptron projection
#         mlp_proj_embd = params["blocks"][b]["mlp"]["c_proj"]["w"].shape[0]
#         gpt.transformer.blocks[b].mlp.projection.build((None, mlp_proj_embd))
#         assert gpt.transformer.blocks[b].mlp.projection.built
#         gpt.transformer.blocks[b].mlp.projection.set_weights([params["blocks"][b]["mlp"]["c_proj"]["w"], params["blocks"][b]["mlp"]["c_proj"]["b"]])    
            
#     gpt.transformer.layer_norm.beta_initializer = tf.keras.initializers.Constant(params["b"])
#     gpt.transformer.layer_norm.gamma_initializer = tf.keras.initializers.Constant(params["g"])
#     gpt.transformer.layer_norm.build((None, None, n_embd))
#     assert gpt.transformer.layer_norm.built    
    

In [5]:
load_weights_into_gpt(gpt2t, params)

In [6]:
x_effort = tf.constant([[6109, 3626, 6100, 345]])
x_effort_emb = gpt2t.embedding(x_effort)
print("x_effort_emb", tldr(x_effort_emb))
assert x_effort_emb.shape == [1, 4, 768]
assert np.allclose(x_effort_emb[0][0][:3], np.array([0.07927368, -0.2979193 ,  0.08817437]), rtol=1e-3, atol=1e-3)

x_effort_emb float32 (1, 4, 768) [ 0.07927368 -0.2979193   0.08817437]


In [7]:
y = gpt2t(x_effort)

In [8]:
assert y.shape == [1, 4, 50257]
assert np.allclose(y[0][0][:3], np.array([-35.52612, -34.92841, -38.39917]), rtol=1e-3, atol=1e-3)

In [9]:
token_ids = generate_text_simple(gpt2t, idx=x_effort, max_new_tokens=5, context_size=256)

In [10]:
assert np.array_equal(token_ids.numpy(), np.array([[6109, 3626, 6100,  345, 2651,   13,  198,  198,  464]]))

In [13]:
# import tiktoken
# tokenizer = tiktoken.get_encoding("gpt2")
start_context = "Every effort moves you"
token_ids = generate_text_simple(
    model=gpt2t,
    idx=text_to_token_ids(start_context),
    max_new_tokens=10,
    # max_new_tokens=30,
    context_size=256
)

print("Output text:\n", token_ids_to_text(token_ids))

Output text:
 Every effort moves you forward.

The first step is to understand
